In [31]:
import numpy as np
from scipy.io import loadmat
import os
import glob
import re
from six.moves import cPickle as pickle

print("import done")

import done


In [ ]:
#get segment from 'patient_segment_result.mat' for file name ordering
def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]

#convert .mat data into ndarray and compress it usingle pickle
def pickle_data(files_pickle, name_pickle):
    print('Reading file ' + files_pickle[0] + ' to ' + files_pickle[-1])
    nrfiles = len(files_pickle)
    placeholder_data = np.ndarray(shape=(nrfiles, 400*10*60, 16), dtype=np.float32)
    placeholder_seq = np.ndarray(shape=(nrfiles), dtype=np.int32)
    for idx, fl in enumerate(files_pickle):
        #open mat file
        mat = loadmat(fl)
        #convert from ndarray object to ndarray
        names = mat['dataStruct'].dtype.names
        ndata = {n: mat['dataStruct'][n][0, 0] for n in names}
        #detect if columns out of order
        if not all(x<y for x, y in zip(ndata['channelIndices'][0], ndata['channelIndices'][0][1:])):
            print('WARNING: Columns out of order in file' + fl)
        placeholder_data[idx, :, :] = ndata['data']
        placeholder_seq[idx] = ndata['sequence']
    #bundle data in a dictionary 
    placeholder = {'data': placeholder_data, 'seq': placeholder_seq}
    
    try:
        with open(name_pickle, 'wb') as f:
            print('Pickling to ' + name_pickle)
            pickle.dump(placeholder, f, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to' + name_pickle + ' :', e)

nrfiles_pickle = 45
        
def dataset_patient(folder_pat, result):
    #get files
    files = sorted(glob.glob(folder_pat + "/*" + result + ".mat"), key=natural_key)
    files = [files[i:i + nrfiles_pickle] for i in xrange(0, len(files), nrfiles_pickle)]
    #concatenate remainder if size too small
    if (len(files[-1]) < 4):
        files[-2].extend(files[-1])
        del files[-1]
    #create pickle files
    for idx, fl in enumerate(files):
        picklefile = folder_pat.split('/')[1] + '_res' + result + '_idx' + str(idx) + '.pickle'
        if (os.path.exists(picklefile)):
            print(picklefile + '  already present. Skip pickling.')
        else:
            pickle_data(fl, picklefile)
    
dataset_patient('./train_1', '1')

print('done')

In [54]:
#convert .mat data into ndarray and compress it usingle pickle
def mat_to_numpy(files):
    print('Reading files ' + files[0] + ' to ' + files[-1])
    nrfiles = len(files)
    placeholder_data = np.ndarray(shape=(nrfiles, 400*10*60, 16), dtype=np.float32)
    for idx, fl in enumerate(files):
        #open mat file
        mat = loadmat(fl)
        #convert from ndarray object to ndarray
        names = mat['dataStruct'].dtype.names
        ndata = {n: mat['dataStruct'][n][0, 0] for n in names}
        #detect if columns out of order
        if not all(x<y for x, y in zip(ndata['channelIndices'][0], ndata['channelIndices'][0][1:])):
            print('WARNING: Columns out of order in file' + fl)
        placeholder_data[idx, :, :] = ndata['data']
    
    return placeholder_data
    

def create_dataset(folder_pat):
    #get files
    files_preictal = glob.glob(folder_pat + "/*1.mat")
    files_interictal = glob.glob(folder_pat + "/*0.mat")
    #randomize read files
    np.random.shuffle(files_preictal)
    np.random.shuffle(files_interictal)

    #read files into ndarrays
    data_preictal = mat_to_numpy(files_preictal[:5])
    labels_preictal = np.ones(data_preictal.shape[0], dtype=np.int32)
    data_interictal = mat_to_numpy(files_interictal[:5])
    labels_interictal = np.ones(data_interictal.shape[0], dtype=np.int32)
    
    data_merge = np.concatenate((data_preictal, data_interictal), axis=0)
    labels_merge = np.concatenate((labels_preictal, labels_interictal), axis=0)
    
    shuffle_idx = np.array(range(data_merge.shape[0]), dtype=np.int32)
    np.random.shuffle(shuffle_idx)
    
    data_merge = data_merge[shuffle_idx, :, :]
    
    print(shuffle_idx)
    
    
#     try:
#         with open(name_pickle, 'wb') as f:
#             print('Pickling to ' + name_pickle)
#             pickle.dump(placeholder, f, pickle.HIGHEST_PROTOCOL)
#     except Exception as e:
#         print('Unable to save data to' + name_pickle + ' :', e)
    
create_dataset('./train_1')



print('done')

Reading files ./train_1/1_130_1.mat to ./train_1/1_50_1.mat
Reading files ./train_1/1_300_0.mat to ./train_1/1_667_0.mat
[0 2 4 7 6 5 8 1 3 9]
done


In [47]:
print(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [20]:
with open('train_1_res1_idx0.pickle', 'rb') as f:
    data = pickle.load(f)
    print(data['data'].shape)
    print(data['seq'].shape)

(37, 240000, 16)
